# The ELS matching procedure

This practical is based on the concepts introduced for optimising electrical contacts in photovoltaic cells. The procedure was published in [J. Mater. Chem. C (2016)]((http://pubs.rsc.org/en/content/articlehtml/2016/tc/c5tc04091d).

<img src="Images/toc.gif">

In this practical we screen electrical contact materials for CH<sub>3</sub>NH<sub>3</sub>PbI<sub>3</sub>. There are three main steps:
* Electronic matching of band energies
* Lattice matching of surface vectors 
* Site matching of under-coordinated surface atoms

# 1. Electronic matching
## Background

Effective charge extraction requires a low barrier to electron or hole transport accross an the interface. This barrier is exponential in the discontinuity of the band energies across the interface. To a first approximation the offset or discontinuity can be estimated by comparing the ionisation potentials (IPs) or electron affinities (EAs) of the two materials, this is known as [Anderson's rule](https://en.wikipedia.org/wiki/Anderson%27s_rule).

<img src="Images/anderson.gif">

Here we have collected a database of 173 measured or estimated semiconductor IPs and EAs (`CollatedData.txt`). We use it as the first step in our screening. The screening is performed by the script `scan_energies.py`. We enforce several criteria:

* The IP and EA of the target material are supplied using the flags `-i` and `-e`
* The IP/EA must be within a certain range from the target material; by default this is set to 0.5 eV, but it can be contolled by the flag `-w`. The window is the full width so the max offset is 0.5*window
* A selective contact should be a semiconductor, so we apply a criterion based on its band gap. If the gap is too large we consider that it would be an insulator. By default this is set to 4.0 eV and is controlled by the flag `-g`

In [ ]:
%%bash
cd Electronic/
python scan_energies.py -h

## Now let's do a proper scan
* IP = 5.7 eV
* EA = 4.0 eV
* Window = 0.25 eV
* Insulating threshold = 4.0 eV

In [ ]:
%%bash
cd Electronic/
python scan_energies.py -i 5.7 -e 4.0 -w 0.5 -g 4.0

# 2. Lattice matching
## Background

For stable interfaces there should be an integer relation between the lattice constants of the two surfaces in contact, which allows for perfect matching, with minimal strain. Generally a strain value of ~ 3% is considered acceptable, above this the interface will be incoherent.

This section uses the [ASE package](https://wiki.fysik.dtu.dk/ase/) to construct the low index surfaces of the materials identified in the electronic step, as well as those of the target material. The code `LatticeMatch.py` to identify optimal matches.

First we need `.cif` files of the materials obtained from the electronic matching. These are obtained from the [Materials Project website](https://www.materialsproject.org). Most of the `.cif` files are there already, but we should add Cu<sub>2</sub>O and GaN, just for practice.

## Lattice matching routine

The lattice matching routine involves obtaining reduced cells for each surface and looking for multiples of each side which match. The procedure is described in more detail in [our paper](http://pubs.rsc.org/en/content/articlehtml/2016/tc/c5tc04091d).

<img src="Images/lattice_match.gif">

The actual clever stuff of the algorithm comes from a paper from Zur and McGill in [J. Appl. Physics (1984)](http://scitation.aip.org/content/aip/journal/jap/55/2/10.1063/1.333084).

<img src="Images/ZurMcGill.jpg">

## The script

The work is done by a python script called `LatticeMatch.py`. As input it reads `.cif` files. It takes a number of flags: 
* `-a` the file containing the crystallographic information of the first material
* `-b` the file containing the crystallographic information of the second material 
* `-s` the strain threshold above which to cutoff, defaults to 0.05
* `-l` the maximum number of times to expand either surface to find matching conditions, defaults to 5

We will run the script in a bash loop to iterate over all interfaces of our contact materials with the (100) and (110) surfaces of pseudo-cubic CH<sub>3</sub>NH<sub>3</sub>PbI<sub>3</sub>. Note that I have made all lattice parameters of CH<sub>3</sub>NH<sub>3</sub>PbI<sub>3</sub> exactly equal, this is to facilitate the removal of duplicate surfaces by the script.

In [ ]:
%%bash
cd Lattice/
for file in *.cif; do python LatticeMatch.py -a MAPI/CH3NH3PbI3.cif -b $file -s 0.03; done

# 3. Site matching

So far the interface matching considered only the magnitude of the lattice vectors. It would be nice to be able to include some measure of how well the dangling bonds can passivate one another. We do this by calculating the site overlap. Basically, we determine the undercoordinated surface atoms on each side and project their positions into a 2D plane. 

<img src="Images/site_overlap.gif">

We then lay the planes over each other and slide them around until there is the maximum coincidence. We calculate the overlap factor from 

$$  ASO = \frac{2S_C}{S_A + S_B}$$

where $S_C$ is the number of overlapping sites in the interface, and $S_A$ and $S_B$ are the number of sites in each surface.

<img src="Images/ASO.gif">

## The script

This section can be run in a stand-alone script called `csl.py`. It relies on a library of the 2D projections of lattice sites from different surfaces, which is called `surface_points.py`. Currently this contains a number of common materials types, but sometimes must be expanded as new materials are identified from the electronic and lattice steps.

`csl.py` takes the following input parameters:
* `-a` The first material to consider
* `-b` The second material to consider
* `-x` The first materials miller index to consider, format : 001
* `-y` The second materials miller index to consider, format : 001
* `-u` The first materials multiplicity, format : 2,2
* `-v` The second materials multiplicity, format : 2,2

We can run it for one example from the previous step, let's say GaN (010)x(2,5) with CH<sub>3</sub>NH<sub>3</sub>PbI<sub>3</sub> (110)x(1,3)


In [ ]:
%%bash
cd Site/
python csl.py -a CH3NH3PbI3 -b GaN -x 110 -y 010 -u 1,3 -v 2,5

## All together

The lattice and site examples above give a a feel for what is going on. For a proper screening procedure it would be nice to be able to run them together. That's exactly what happens with the `LatticeSite.py` script. It uses a new class `Pair` to store and pass information about the interface pairings. This includes the materials names, miller indices of matching surfaces, strians, multiplicities etc.

The `LatticeSite.py` script takes the same variables as `LatticeMatch.py`. It just takes a little longer to run, so a bit of patience is required.

This script outputs the standard pair information as well as the site matching factor, which is calculated as

$$ \frac{100\times ASO}{1 + |\epsilon|}$$

where the $ASO$ was defined above, and $\epsilon$ in the average of the $u$ and $v$ strains. The number is a measure of the mechanical stability of an interface. A perfect interface of a material with itself would have a fator of 100.

Where lattices match but no information on the structure of the surface exists it is flagged up. You can always add new surfaces as required.

In [ ]:
%%bash
cd Site/
for file in *cif; do python LatticeSite.py -a MAPI/CH3NH3PbI3.cif -b $file -s 0.03; done